# 最终的优化版本（特征工程和模型融合）
    1. 数据预处理
            1.1 判断数据是否合规
            1.2 缺失值的检验 
                统计特征填补（众数、均值、中位数）
            1.3 字段类型的探索
            1.4 连续值分箱(可处理某些异常值和相当于对于某些线性的变量引入了非线性变换，如：激活函数。 但会破环数据的正态分布)
    特征衍生
    特征筛选

In [536]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from tqdm import tqdm


In [537]:
data_train = pd.read_csv('./data/train.csv')
data_test_a = pd.read_csv('./data/testA.csv')
print(data_train.shape, data_test_a.shape)

(800000, 47) (200000, 46)


In [538]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 47 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  800000 non-null  int64  
 1   loanAmnt            800000 non-null  float64
 2   term                800000 non-null  int64  
 3   interestRate        800000 non-null  float64
 4   installment         800000 non-null  float64
 5   grade               800000 non-null  object 
 6   subGrade            800000 non-null  object 
 7   employmentTitle     799999 non-null  float64
 8   employmentLength    753201 non-null  object 
 9   homeOwnership       800000 non-null  int64  
 10  annualIncome        800000 non-null  float64
 11  verificationStatus  800000 non-null  int64  
 12  issueDate           800000 non-null  object 
 13  isDefault           800000 non-null  int64  
 14  purpose             800000 non-null  int64  
 15  postCode            799999 non-nul

In [539]:
# 1.1. 判断数据是否合规 （主要是看数据id是否重复和重复列） --> 是否和官方字典是否保持一致
data_train['id'].nunique() == data_train.shape[0]

True

In [540]:
# 查看数据是否重复
data_train.duplicated().sum()

0

In [541]:
# 1.2 缺失值的检验
# 但此结果只代表没有None或者Nan，并不排除可能存在别的值来表示缺失值的情况，因此后面我们要对其进一步分析
def missing (data):
    """
      计算每一列的缺失值的占比 
    """
    missing_number = data.isnull().sum().sort_values(ascending=False)
    missing_percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number,missing_percent], axis=1, keys=['Missing_number', 'Missing_percent'])

    return missing_values
  
missing_trian = missing(data_train)
missing_trian


,Missing_number,Missing_percent
n11,69752,0.087190
employmentLength,46799,0.058499
n8,40271,0.050339
n7,40270,0.050338
n1,40270,0.050338
n2,40270,0.050338
n3,40270,0.050338
n5,40270,0.050338
n6,40270,0.050338
n14,40270,0.050338


In [542]:
data_train.describe()

,id,loanAmnt,term,interestRate,installment,employmentTitle,homeOwnership,annualIncome,verificationStatus,isDefault,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
count,800000.000000,800000.000000,800000.000000,800000.000000,800000.000000,799999.000000,800000.000000,8.000000e+05,800000.000000,800000.000000,...,759730.000000,759730.000000,759730.000000,759729.000000,759730.000000,766761.000000,730248.000000,759730.000000,759730.000000,759730.000000
mean,399999.500000,14416.818875,3.482745,13.238391,437.947723,72005.351714,0.614213,7.613391e+04,1.009683,0.199513,...,8.107937,8.575994,8.282953,14.622488,5.592345,11.643896,0.000815,0.003384,0.089366,2.178606
std,230940.252013,8716.086178,0.855832,4.765757,261.460393,106585.640204,0.675749,6.894751e+04,0.782716,0.399634,...,4.799210,7.400536,4.561689,8.124610,3.216184,5.484104,0.030075,0.062041,0.509069,1.844377
min,0.000000,500.000000,3.000000,5.310000,15.690000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,199999.750000,8000.000000,3.000000,9.750000,248.450000,427.000000,0.000000,4.560000e+04,0.000000,0.000000,...,5.000000,4.000000,5.000000,9.000000,3.000000,8.000000,0.000000,0.000000,0.000000,1.000000
50%,399999.500000,12000.000000,3.000000,12.740000,375.135000,7755.000000,1.000000,6.500000e+04,1.000000,0.000000,...,7.000000,7.000000,7.000000,13.000000,5.000000,11.000000,0.000000,0.000000,0.000000,2.000000
75%,599999.250000,20000.000000,3.000000,15.990000,580.710000,117663.500000,1.000000,9.000000e+04,2.000000,0.000000,...,11.000000,11.000000,10.000000,19.000000,7.000000,14.000000,0.000000,0.000000,0.000000,3.000000
max,799999.000000,40000.000000,5.000000,30.990000,1715.420000,378351.000000,5.000000,1.099920e+07,2.000000,1.000000,...,70.000000,132.000000,79.000000,128.000000,45.000000,82.000000,4.000000,4.000000,39.000000,30.000000


In [543]:
missing_trian = missing(data_train)
missing_trian

,Missing_number,Missing_percent
n11,69752,0.087190
employmentLength,46799,0.058499
n8,40271,0.050339
n7,40270,0.050338
n1,40270,0.050338
n2,40270,0.050338
n3,40270,0.050338
n5,40270,0.050338
n6,40270,0.050338
n14,40270,0.050338


In [544]:
# 1.3 数据类型的探索及处理
# 1.3.1 离散特征字段和数值型字段
category_fea = ['grade', 'subGrade', 'employmentTitle', 'homeOwnership', 'verificationStatus', 'purpose', 'postCode', 'regionCode', \
                 'applicationType', 'initialListStatus', 'title', 'policyCode']
numerical_fea = list(filter(lambda x: x not in category_fea, list(data_train.columns)))

label = 'isDefault'

#还有另外两个比较特别的时间序列

numerical_fea.remove(label)

In [545]:
# 1.3.2 时间对象的处理
startdate = pd.to_datetime(data_train['issueDate'].min(), format='%Y-%m-%d')

for data in [data_train, data_test_a]:
    data['issueDate'] = pd.to_datetime(data['issueDate'], format='%Y-%m-%d')
    #构造时间特征
    data['issueDateDT'] = data['issueDate'].apply(lambda x: x-startdate).dt.days
    data['issueDate_year'] = data['issueDate'].dt.year
    data['issueDate_month'] = data['issueDate'].dt.month
    # data_train['issueDate_day'].value_counts() # 唯一值，所以可以删去
    data.drop('issueDate', axis=1, inplace=True)


In [546]:
def employmenLength_to_int(s):
    if pd.isnull(s):
        return s
    else: 
        return np.int8(s.split()[0])

for data in [data_train, data_test_a]:
    data['employmentLength'].replace('10+ years', "10 years", inplace=True)
    data['employmentLength'].replace('< 1 year', "1 years", inplace=True)
    data['employmentLength'].replace(np.nan, '1 years', inplace=True)
    data['employmentLength'].replace(np.inf, '10 years', inplace=True)
    data['employmentLength'] = data['employmentLength'].apply(lambda x: employmenLength_to_int(x))

In [547]:
# 1.3.3 部分类别特征
cate_features = ['grade', 'subGrade', 'employmentTitle', 'homeOwnership', 'verificationStatus', 'purpose', 'postCode', 'regionCode', \
                 'applicationType', 'initialListStatus', 'title', 'policyCode']
                 
data = pd.concat([data_train,data_test_a], axis=0)

for f in cate_features:
    print(f, '类型数：', data[f].nunique())

grade 类型数： 7
subGrade 类型数： 35
employmentTitle 类型数： 298101
homeOwnership 类型数： 6
verificationStatus 类型数： 3
purpose 类型数： 14
postCode 类型数： 935
regionCode 类型数： 51
applicationType 类型数： 2
initialListStatus 类型数： 2
title 类型数： 47903
policyCode 类型数： 1


In [548]:
# policyCode 类型数： 1 直接删除
for data in [data_train, data_test_a]:
    data.drop('policyCode', axis=1, inplace=True)

# regionCode  已经是编码了所以可直接跳过

In [549]:
###  使用之前首先处理掉非数值型特征
## 缺失值处理 -> 并不是指np.nan 而是那些无法溯源的值。例如：有些缺失值会用其他符号标记
# 不处理或者简单处理
# 业务溯源
# 简单标记（适用于缺失值占比0.0几%）
# 统计插补（（适用于缺失值占比0.0几%）
# 分层插补
# 热平台插补（类似于K临近）
# 差值法（牛顿插补、拉格朗日插补、多重插补）
# for col in data_train.columns:
#     if data_train[col].isnull().sum() > 0:
#         data_train[col] = data_train[col].fillna(data_train[col].std)

# for col in data_test_a.columns:
#     if data_test_a[col].isnull().sum() > 0:
#         data_test_a[col] = data_test_a[col].fillna(data_test_a[col].std)

# for data in [data_train, data_test_a]:
#     for col in data_test_a.columns:
#         data[col].replace(np.nan, 0, inplace=True)
#         data[col].replace(np.inf, 0, inplace=True)


def col_null(data, col):
    """
    单独处理某一列的空值 
    """
    for i, col in enumerate(col):
        data[col].replace(np.nan, 0, inplace=True)
        data[col].replace(np.inf, 0, inplace=True)
        
        if data[col].isnull().sum() > 0:
            data_train[col].fillna(data[col].std)

    return data
        


In [550]:
# 1.3.4 one-hot 独热编码（小样本+五顺序意义）和 get.dumnps() 作用差不多
# id gender         id gender_M  gender_F
# 1   M       ->     1    1        0
# 2   F              2    0        1

def cate_colName(Transformer, category_cols, drop='if_bianry'):
    """
    离散字段独热编码后字段命名函数
    ：param Transformer: 独热编码转换器
    ：parame category_cols: 输入转换器的离散变量
    ：param drop: 是否具有二分类参数（二分类不需要进行编码）
    """
    cate_col_new = []
    col_value = Transformer.categories_

    for i, j in enumerate(category_cols):
        if (drop == 'if_bianry') and len(col_value[i]) == 2:
            cate_col_new.append(j)
        else:
            for f in col_value[i]:
                feture_name = j + '_' + str(f)
                cate_col_new.append(feture_name)

    return cate_col_new


In [551]:
cate_features_no_sequence = ['homeOwnership','verificationStatus', 'purpose']

data = pd.concat([data_train, data_test_a], axis=0)

hot_data = pd.DataFrame(data,columns=cate_features_no_sequence)

ohe = OneHotEncoder()
ohe.fit_transform(hot_data)

hot_all = pd.DataFrame(ohe.transform(hot_data).toarray(), columns=cate_colName(ohe, cate_features_no_sequence))

In [552]:
data_train.shape

(800000, 48)

In [553]:
# 分离 train_hot 和 test_hot

hot_train = hot_all.loc[:data_train.shape[0]-1, :]
data_train = pd.concat([data_train, hot_train], axis=1, join='outer')

data_train


,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,purpose_4,purpose_5,purpose_6,purpose_7,purpose_8,purpose_9,purpose_10,purpose_11,purpose_12,purpose_13
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,3000.0,3,12.99,101.07,C,C2,54.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,799995,25000.0,3,14.49,860.41,C,C4,2659.0,7,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
799996,799996,17000.0,3,7.90,531.94,A,A4,29205.0,10,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
799997,799997,6000.0,3,13.33,203.12,C,C3,2582.0,10,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
799998,799998,19200.0,3,6.92,592.14,A,A4,151.0,10,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [554]:
hot_test = hot_all.loc[data_train.shape[0]:, :]
hot_test = hot_test.reset_index(drop=True)
data_test_a = pd.concat([data_test_a, hot_test], axis=1, join='outer')

data_test_a

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,purpose_4,purpose_5,purpose_6,purpose_7,purpose_8,purpose_9,purpose_10,purpose_11,purpose_12,purpose_13
0,800000,14000.0,3,10.99,458.28,B,B3,7027.0,10,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,800001,20000.0,5,14.65,472.14,C,C5,60426.0,10,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,800002,12000.0,3,19.99,445.91,D,D4,23547.0,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,800003,17500.0,5,14.31,410.02,C,C4,636.0,4,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,800004,35000.0,3,17.09,1249.42,D,D1,368446.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,999995,7000.0,3,11.14,229.64,B,B2,330967.0,7,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199996,999996,6000.0,3,6.24,183.19,A,A2,38930.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199997,999997,14000.0,5,15.88,339.57,C,C4,282016.0,8,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199998,999998,8000.0,3,18.06,289.47,D,D2,97.0,4,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [555]:
# 1.3.4  encoder 或者自映射编码(作用于顺序序列，且取值范围较多)
for data in [data_train, data_test_a]:
    data['grade'] = data['grade'].map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7})

le = LabelEncoder()
# 高维类别特征需要进行转换
for col in tqdm(['employmentTitle', 'postCode', 'title','subGrade']):
    le.fit(list(data_train[col].astype(str).values) + list(data_test_a[col].astype(str).values))
    data_train[col] = le.transform(list(data_train[col].astype(str).values))
    data_test_a[col] = le.transform(list(data_test_a[col].astype(str).values))
print('Label Encoding 完成')

100%|██████████| 4/4 [00:07<00:00,  1.95s/it]

Label Encoding 完成


In [556]:
print(data_train.shape, data_test_a.shape)

(800000, 71) (200000, 70)


In [557]:
data_test_a['grade']

0         2
1         3
2         4
3         3
4         4
         ..
199995    2
199996    1
199997    3
199998    4
199999    1
Name: grade, Length: 200000, dtype: int64

In [558]:
missing_trian02 = missing(data_train)
missing_trian02

,Missing_number,Missing_percent
n11,69752,0.087190
n8,40271,0.050339
n5,40270,0.050338
n13,40270,0.050338
n9,40270,0.050338
...,...,...
totalAcc,0,0.000000
initialListStatus,0,0.000000
applicationType,0,0.000000
earliesCreditLine,0,0.000000


In [559]:
data_train

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,purpose_4,purpose_5,purpose_6,purpose_7,purpose_8,purpose_9,purpose_10,purpose_11,purpose_12,purpose_13
0,0,35000.0,5,19.52,917.97,5,21,192025,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,18000.0,5,18.49,461.90,4,16,104733,5,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,12000.0,5,16.99,298.17,4,17,189509,8,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,11000.0,3,7.26,340.96,1,3,249631,10,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,3000.0,3,12.99,101.07,3,11,256267,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,799995,25000.0,3,14.49,860.41,3,13,145357,7,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
799996,799996,17000.0,3,7.90,531.94,1,3,168143,10,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
799997,799997,6000.0,3,13.33,203.12,3,12,138795,10,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
799998,799998,19200.0,3,6.92,592.14,1,3,45602,10,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [560]:
missing_trian = missing(data_train)
missing_trian

,Missing_number,Missing_percent
n11,69752,0.087190
n8,40271,0.050339
n5,40270,0.050338
n13,40270,0.050338
n9,40270,0.050338
...,...,...
totalAcc,0,0.000000
initialListStatus,0,0.000000
applicationType,0,0.000000
earliesCreditLine,0,0.000000


In [561]:
data_train.columns

Index(['id', 'loanAmnt', 'term', 'interestRate', 'installment', 'grade',
       'subGrade', 'employmentTitle', 'employmentLength', 'homeOwnership',
       'annualIncome', 'verificationStatus', 'isDefault', 'purpose',
       'postCode', 'regionCode', 'dti', 'delinquency_2years', 'ficoRangeLow',
       'ficoRangeHigh', 'openAcc', 'pubRec', 'pubRecBankruptcies', 'revolBal',
       'revolUtil', 'totalAcc', 'initialListStatus', 'applicationType',
       'earliesCreditLine', 'title', 'n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6',
       'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'issueDateDT',
       'issueDate_year', 'issueDate_month', 'homeOwnership_0',
       'homeOwnership_1', 'homeOwnership_2', 'homeOwnership_3',
       'homeOwnership_4', 'homeOwnership_5', 'verificationStatus_0',
       'verificationStatus_1', 'verificationStatus_2', 'purpose_0',
       'purpose_1', 'purpose_2', 'purpose_3', 'purpose_4', 'purpose_5',
       'purpose_6', 'purpose_7', 'purpose_8', 'purpose_9', 'purp

In [562]:
# 聚类分箱


serious_col = ['interestRate','installment','employmentLength', 'annualIncome', 'dti',
               'delinquency_2years', 'openAcc', 'pubRec', 'pubRecBankruptcies', 'revolBal',
               'revolUtil', 'totalAcc']

data_all = pd.concat([data_train, data_test_a], axis=0)

KB_data = pd.DataFrame(data_all, columns=serious_col)

data_all = col_null(KB_data, serious_col)

dis = KBinsDiscretizer(n_bins=2, encode='ordinal', strategy='kmeans')
dis.fit_transform(KB_data)

kb_data = pd.DataFrame(dis.transform(KB_data), columns=['interestRate_kb','installment_kb','employmentLength_kb', 'annualIncome_kb', 'dti_kb',
                                            'delinquency_2years_kb', 'openAcc_kb', 'pubRec_kb', 'pubRecBankruptcies_kb', 'revolBal_kb',
                                            'revolUtil_kb', 'totalAcc_kb'])




In [563]:
data_train['dti']

0         17.05
1         27.83
2         22.77
3         17.21
4         32.16
          ...  
799995    19.03
799996    15.72
799997    12.11
799998    29.25
799999     8.99
Name: dti, Length: 800000, dtype: float64

In [564]:
# 分离 train_hot 和 test_hot

kb_train = kb_data.loc[:data_train.shape[0]-1, :]
data_train = pd.concat([data_train, kb_train], axis=1, join='outer')

kb_test = kb_data.loc[data_train.shape[0]:, :].reset_index(drop=True)
data_test_a = pd.concat([data_test_a, kb_test], axis=1, join='outer')


In [565]:
print(data_train.shape, data_test_a.shape)

(800000, 83) (200000, 82)


### 特征衍生

In [566]:
def Bianry_Group_Statistics(keycol,
                            features,
                            col_num=None,
                            col_cat=None,
                            num_stat=['mean', 'var', 'max', 'min', 'median'],
                            cat_stat=['mean', 'var', 'max', 'min', 'median', 'count', 'nunique'],
                            flag = None,
                            quant=True):
    """ 
    双变量分组统计特征衍生
    ：param keycol: 分组参考的关键变量
    ：param features： 原始数据集
    ：param col_num: 参与衍生的连续型变量
    ：param col_cat: 参与衍生的离散值变量
    ：param num_stat: 连续型变量分组统计
    ：param cat_stat: 离散型变量分组统计
    ：param quant: 是否计算分位数
    ：param flag: 数据标记

    ：return：交叉衍生后的新特征和新特征名称
    """
     
    print("-------------开始一阶特征衍生-----------------")
    # 当输入的有连续型特征：
    if col_num != None:
        aggs_num = {}
        colNames = col_num

        # 创建agg方法所需字典
        for col in col_num: 
            aggs_num[col] = num_stat
        # 创建衍生特征的名称列表
        cols_num = [keycol]
        for key in aggs_num.keys():
            cols_num.extend([key+'_'+keycol+'_'+stat for stat in aggs_num[key]])
        # 创建衍生特征

        features_num_new = features[[flag]+[keycol]+col_num].groupby(keycol).agg(aggs_num).reset_index()
        features_num_new.columns = cols_num
        

        # 当输入的有离散型特征：
        if col_cat != None:
            print("--------------连续型+离散型--------------")
            aggs_cat = {}
            colNames = col_cat + col_num

            # 创建agg方法所需字典
            for col in col_cat: 
                aggs_cat[col] = cat_stat
            # 创建衍生特征的名称列表
            cols_cat = [keycol]
            for key in aggs_cat.keys():
                cols_cat.extend([key+'_'+keycol+'_'+stat for stat in aggs_cat[key]])
            # 创建衍生特征
            features_cat_new = features[[flag]+col_cat+[keycol]].groupby(keycol).agg(aggs_cat).reset_index()
            features_cat_new.columns = cols_cat
          
            # 合并连续型变量和离散型变量衍生结果

            features_temp = pd.merge(features_num_new, features_cat_new, how='left', on=keycol)
            features_new = pd.merge(features[[flag]+[keycol]], features_temp, how='left', on=keycol)
            colNames_new = cols_num + cols_cat ## 每个都会包含一个Keycol, 所以会有两个。

            colNames_new.remove(keycol)
            colNames_new.remove(keycol)


        ## 当只有连续型变量时
        else: 
            print("--------------只有连续型--------------")
            # merge 连续变量的衍生结果和原始数据，
            features_new = pd.merge(features[[keycol]+[flag]], features_num_new, how='left', on=keycol)  
            colNames_new = cols_num 
            colNames_new.remove(keycol)

    #当输入只有离散变量时
    else:
        if col_cat != None:
            print("--------------只有离散型--------------")
            aggs_cat = {}
            colNames = col_cat

            # 创建agg方法所需字典
            for col in col_cat: 
                aggs_cat[col] = cat_stat
            # 创建衍生特征的名称列表
            cols_cat = [keycol]
            for key in aggs_cat.keys():
                cols_cat.extend([key+'_'+keycol+'_'+stat for stat in aggs_cat[key]])
            # 创建衍生特征
            features_cat_new = features[col_cat+[keycol]].groupby(keycol).agg(aggs_cat).reset_index()
            features_cat_new.columns = cols_cat
            features_new = pd.merge(features[[keycol]+[flag]], features_cat_new, how='left', on=keycol)
            colNames_new = cols_cat
            colNames_new.remove(keycol)
 
    
    if quant:
        print("--------------上下四分位数--------------")
        ## 分位数 agg() 并没有集成，所以要手写
        def q1(x):
            """
            下四分位数 
            """
            return x.quantile(0.25)

        def q2(x):
            """
            下四分位数 
            """
            return x.quantile(0.75)

        aggs = {}
        for col in colNames:
            aggs[col] = ['q1', 'q2']
        
        cols = [keycol]
        for key in aggs.keys():
            cols.extend([key+'_'+keycol+'_'+stat for stat in aggs[key]])
        
        aggs = {}
        for col in colNames:
            aggs[col] = [q1, q2]

        features_temp = features[colNames+[keycol]].groupby(keycol).agg(aggs).reset_index()
        features_temp.columns = cols

        features_new = pd.merge(features_new, features_temp, how='left', on=keycol)
        colNames_new = colNames_new + cols
        colNames_new.remove(keycol) 
    
    features_new.drop([keycol], axis=1, inplace=True)

    return features_new, colNames_new



In [567]:
d1 = pd.DataFrame({'id':[0,1,2,3,4,5],'tenure':[1,2,1,3,2,3], 'x1':[2,5,1,2,6,1], 'x2':[2, 5,1 ,2, 6,1]})
d1

,id,tenure,x1,x2
0,0,1,2,2
1,1,2,5,5
2,2,1,1,1
3,3,3,2,2
4,4,2,6,6
5,5,3,1,1


In [568]:
col_cat = ['x1']
col_num = ['x2']
keycol = 'tenure'
df, col  = Bianry_Group_Statistics(keycol, features=d1, col_cat=col_cat, col_num=col_num, flag='id')

-------------开始一阶特征衍生-----------------
--------------连续型+离散型--------------
--------------上下四分位数--------------


In [569]:
df

,id,x2_tenure_mean,x2_tenure_var,x2_tenure_max,x2_tenure_min,x2_tenure_median,x1_tenure_mean,x1_tenure_var,x1_tenure_max,x1_tenure_min,x1_tenure_median,x1_tenure_count,x1_tenure_nunique,x1_tenure_q1,x1_tenure_q2,x2_tenure_q1,x2_tenure_q2
0,0,1.5,0.5,2,1,1.5,1.5,0.5,2,1,1.5,2,2,1.25,1.75,1.25,1.75
1,1,5.5,0.5,6,5,5.5,5.5,0.5,6,5,5.5,2,2,5.25,5.75,5.25,5.75
2,2,1.5,0.5,2,1,1.5,1.5,0.5,2,1,1.5,2,2,1.25,1.75,1.25,1.75
3,3,1.5,0.5,2,1,1.5,1.5,0.5,2,1,1.5,2,2,1.25,1.75,1.25,1.75
4,4,5.5,0.5,6,5,5.5,5.5,0.5,6,5,5.5,2,2,5.25,5.75,5.25,5.75
5,5,1.5,0.5,2,1,1.5,1.5,0.5,2,1,1.5,2,2,1.25,1.75,1.25,1.75


In [570]:
col

['x2_tenure_mean',
 'x2_tenure_var',
 'x2_tenure_max',
 'x2_tenure_min',
 'x2_tenure_median',
 'x1_tenure_mean',
 'x1_tenure_var',
 'x1_tenure_max',
 'x1_tenure_min',
 'x1_tenure_median',
 'x1_tenure_count',
 'x1_tenure_nunique',
 'x1_tenure_q1',
 'x1_tenure_q2',
 'x2_tenure_q1',
 'x2_tenure_q2']

grade: 类型数 ：7 </br>
subGrade: 类型数 ：35</br>
employmentTitle: 类型数 ：79282 </br>
homeOwnership: 类型数 ：6 </br>
verificationStatus: 类型数 ：3 </br>
purpose: 类型数 ：14 </br>
postCode: 类型数 ：889 </br>
regionCode: 类型数 ：51 </br>
applicationType: 类型数 ：2 </br>
initialListStatus: 类型数 ：2 </br>
title: 类型数 ：12058</br>
policyCode: 类型数 ：1 </br>

In [571]:
missing_trian = missing(data_train)
missing_trian

,Missing_number,Missing_percent
n11,69752,0.087190
n8,40271,0.050339
n9,40270,0.050338
n0,40270,0.050338
n1,40270,0.050338
...,...,...
loanAmnt,0,0.000000
issueDateDT,0,0.000000
issueDate_year,0,0.000000
issueDate_month,0,0.000000


In [572]:
data_train.shape

(800000, 83)

In [573]:
keycol = 'regionCode'
col_nums = ['dti', 'annualIncome']

creat_features, cols = Bianry_Group_Statistics(keycol=keycol, features=data_train, col_num=col_nums, flag='id')
data_train = pd.merge(data_train, creat_features, how='left', on='id')

creat_features, cols = Bianry_Group_Statistics(keycol=keycol, features=data_test_a, col_num=col_nums, flag='id')
data_test_a = pd.merge(data_test_a, creat_features, how='left', on='id')

-------------开始一阶特征衍生-----------------
--------------只有连续型--------------
--------------上下四分位数--------------
-------------开始一阶特征衍生-----------------
--------------只有连续型--------------
--------------上下四分位数--------------


In [574]:
print(data_train.shape, data_test_a.shape)

(800000, 97) (200000, 96)


In [575]:
cat_olumns = ['interestRate_kb','installment_kb','employmentLength_kb', 'annualIncome_kb', 'dti_kb'\
              'delinquency_2years_kb', 'openAcc_kb', 'pubRec_kb', 'pubRecBankruptcies_kb', 'revolBal_kb',\
              'revolUtil_kb', 'totalAcc_kb']

In [576]:
data_train['dti_kb']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
799995    0.0
799996    0.0
799997    0.0
799998    0.0
799999    0.0
Name: dti_kb, Length: 800000, dtype: float64

In [577]:
keycol = 'dti_kb'
col_nums = ['annualIncome','delinquency_2years', 'openAcc', 'pubRec', 'totalAcc']
col_cats = [ 'openAcc_kb',  'pubRecBankruptcies_kb', 'revolBal_kb','revolUtil_kb']

creat_features, cols = Bianry_Group_Statistics(keycol=keycol, features=data_train, col_num=col_nums, col_cat=col_cats, flag='id')
data_train = pd.merge(data_train, creat_features, how='left', on='id')

print("  hh  我结束了")

creat_features, cols = Bianry_Group_Statistics(keycol=keycol, features=data_test_a, col_num=col_nums, col_cat=col_cats, flag='id')
data_test_a = pd.merge(data_test_a, creat_features, how='left', on='id')

-------------开始一阶特征衍生-----------------
--------------连续型+离散型--------------
--------------上下四分位数--------------
  hh  我结束了
-------------开始一阶特征衍生-----------------
--------------连续型+离散型--------------
--------------上下四分位数--------------


In [578]:
print(data_train.shape, data_test_a.shape)

(800000, 168) (200000, 167)


In [579]:
keycol = 'annualIncome_kb'
col_nums = ['delinquency_2years', 'openAcc', 'pubRec', 'totalAcc']
col_cats = [ 'openAcc_kb',  'pubRecBankruptcies_kb', 'revolBal_kb','revolUtil_kb']

creat_features, cols = Bianry_Group_Statistics(keycol=keycol, features=data_train, col_num=col_nums, col_cat=col_cats, flag='id')
data_train = pd.merge(data_train, creat_features, how='left', on='id')

creat_features, cols = Bianry_Group_Statistics(keycol=keycol, features=data_test_a, col_num=col_nums, col_cat=col_cats, flag='id')
data_test_a = pd.merge(data_test_a, creat_features, how='left', on='id')

-------------开始一阶特征衍生-----------------
--------------连续型+离散型--------------
--------------上下四分位数--------------
-------------开始一阶特征衍生-----------------
--------------连续型+离散型--------------
--------------上下四分位数--------------


In [580]:
print(data_train.shape, data_test_a.shape)

(800000, 232) (200000, 231)
